In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
df_working = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

RANDOM_SEED=42

def prepare_age(df):
    # Fill in missing Age values from distribution of present Age values 
    mean = df["Age"].mean()
    std = df["Age"].std()
    is_null = df["Age"].isnull().sum()
    # compute enough (== is_null().sum()) random numbers between the mean, std
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = df["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    df["Age"] = age_slice
    df["Age"] = df["Age"].astype(int)
    
    # Quantize age into 5 classes
    df['Age_Group'] = pd.qcut(df['Age'],5, labels=False)
    df.drop(['Age'], axis=1, inplace=True)
    return df

def prepare_fare(df):
    df['Fare'].fillna(0, inplace=True)
    df['Fare_Group'] = pd.qcut(df['Fare'],5,labels=False)
    df.drop(['Fare'], axis=1, inplace=True)
    return df 

def prepare_genders(df):
    genders = {"male": 0, "female": 1, "unknown": 2}
    df['Sex'] = df['Sex'].map(genders)
    df['Sex'].fillna(2, inplace=True)
    df['Sex'] = df['Sex'].astype(int)
    return df

def prepare_embarked(df):
    df['Embarked'].replace('', 'U', inplace=True)
    df['Embarked'].fillna('U', inplace=True)
    ports = {"S": 0, "C": 1, "Q": 2, "U": 3}
    df['Embarked'] = df['Embarked'].map(ports)
    return df
    


dataframe = prepare_embarked(prepare_genders(prepare_fare(prepare_age(df_working))))


array = dataframe.values
 
X = np.array(array)[:,1:8].astype('float32')
Y = np.array(array)[:,0].astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(X)
 
test_size = 0.33
seed = 7
 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size,
random_state=seed)

In [4]:
import sagemaker
import boto3

#Create a SageMaker session
sagemaker_session = sagemaker.Session()
bucket = "linear-titanic"
prefix = "linear-learner" #prefix is a sub-folder/key within the S3 bucket

#Access SageMaker role created prior to session
#Need to pass role to training job
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::589173826352:role/service-role/AmazonSageMaker-ExecutionRole-20211101T170735


In [5]:
#Need to convert dataset to RecordIO format for Linear Learner to understand
import io 
import sagemaker.amazon.common as smac 
import os

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, X_train, Y_train)
buf.seek(0) 

###Uploading training data
#Filename for training data we are uploading to S3 
key = 'linear-train-data'
#Upload training data to S3
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

###Uploading test data
buf = io.BytesIO() # create an in-memory byte array (buf is a buffer I will be writing to)
smac.write_numpy_to_dense_tensor(buf, X_test, Y_test)
buf.seek(0)

#Sub-folder for test data
key = 'linear-test-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test', key)).upload_fileobj(buf)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_test_data))

###Model Artifacts
output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('Training artifacts will be uploaded to: {}'.format(output_location))

uploaded training data location: s3://linear-titanic/linear-learner/train/linear-train-data
uploaded training data location: s3://linear-titanic/linear-learner/test/linear-test-data
Training artifacts will be uploaded to: s3://linear-titanic/linear-learner/output


In [6]:
from sagemaker.amazon.amazon_estimator import image_uris
container = image_uris.retrieve('linear-learner', boto3.Session().region_name)

linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       instance_count = 1, 
                                       instance_type = 'ml.c4.xlarge',
                                       output_path = output_location,
                                       sagemaker_session = sagemaker_session)

In [8]:
linear.set_hyperparameters(feature_dim = 7,
                           predictor_type = 'binary_classifier',
                           num_classes = 1,
                           mini_batch_size = 20,
                           epochs = 5,
                           num_models = 10,
                           loss = 'auto')

In [9]:
#Pass in S3 training_data path variable we declared earlier
linear.fit({'train': s3_train_data})

2021-11-01 22:47:53 Starting - Starting the training job...
2021-11-01 22:48:16 Starting - Launching requested ML instancesProfilerReport-1635806873: InProgress
...
2021-11-01 22:48:43 Starting - Preparing the instances for training.........
2021-11-01 22:50:18 Downloading - Downloading input data...
2021-11-01 22:50:36 Training - Downloading the training image.Docker entrypoint called with argument(s): train
Running default environment configuration script
[11/01/2021 22:50:59 INFO 140137837922112] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'au

In [10]:
linear_classifier = linear.deploy(initial_instance_count = 1,
                                          instance_type = 'ml.m4.xlarge')

#need to make sure data is in correct format for deployed model
from sagemaker.predictor import csv_serializer, json_deserializer
linear_classifier.serializer = csv_serializer
linear_classifier.deserializer = json_deserializer

------!

In [14]:
#result = linear_classifier.predict(X_test)
#result #should be a JSON

#Iterate the result JSON to get an NP array of all the predictions so we can compare to Y test
predictions = np.array([res['predicted_label'] for res in result['predictions']])
predictions #should now be an numpy array

#Visualize how accurate predictions are relative to y_test
#plt.scatter(Y_test, predictions)

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,

In [15]:
from sklearn import metrics 
#metrics.mean_squared_error(Y_test, predictions))

In [16]:
metrics.balanced_accuracy_score(Y_test, predictions)

0.8403472931562819

In [18]:
metrics.accuracy_score(Y_test, predictions)

0.8541666666666666

In [21]:
print(metrics.classification_report(Y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.87      0.90      0.88       267
         1.0       0.83      0.78      0.80       165

    accuracy                           0.85       432
   macro avg       0.85      0.84      0.84       432
weighted avg       0.85      0.85      0.85       432

